In [1]:
import numpy as np

# Comparison of Extragradient and Goldstein-Levitin-Polyak Methods for Variational Inequalities

## Introduction

Variational Inequalities (VIs) are powerful mathematical frameworks for modeling equilibrium problems. This document compares two fundamental methods for solving VIs:

- Goldstein-Levitin-Polyak (GLP) Projection Method

- Extragradient (EG) Method

Problem Formulation
Given:

- Feasible set: $C = \{(x,y)| x\in[-1,1], y\in[-1,1] \}$

- Operator: $F(x, y) = (-y, x)$

- Solution: $(x^*, y^*) = (0, 0)$

## Algorithm Comparison
### Goldstein-Levitin-Polyak (GLP) Method

$x_{k} = P_{C}(x_k - \gamma F(x_k))$

Convergence Requirements:

- Strongly monotone operator

- Step size: $\gamma \in (0, 2\mu/L^2)$

Limitations:

- Fails for merely monotone operators

- Prone to oscillations and divergence

## Extragradient Method

Prediction:

$\hat{x}_{k} = P_{C}(x_k - \gamma F(x_k))$

Correction:

$x_{k+1} = P_{C}(x_k - \gamma F(\hat{x}_k))$

Where $P_C$ is the projection onto the feasible set $C$.

Convergence Requirements:

- Monotone + Lipschitz operator

- Step size: $\gamma \in (0, 1/L)$

Advantages:

- Handles merely monotone operators

- Robust convergence behavior

- Prevents oscillation through prediction step

In [2]:
# Projection onto box constraint [-1, 1] x [-1, 1]
def project_C(x):
    return np.clip(x, -1.0, 1.0)

# Operator F(x) = (-y, x) for input vector (x, y)
def F(x):
    return np.array([-x[1], x[0]])

# Goldstein-Levitin-Polyak (GLP) Projection Method
def glp_projection(x0, gamma, max_iter=20):
    x = np.array(x0, dtype=float)
    history = [x.copy()]
    
    for _ in range(max_iter):
        x_next = project_C(x - gamma * F(x))
        x = x_next
        history.append(x.copy())
        
    return np.array(history)

# Extragradient Method
def extragradient(x0, gamma, max_iter=20):
    x = np.array(x0, dtype=float)
    history = [x.copy()]
    
    for _ in range(max_iter):
        # Prediction step
        y_hat = project_C(x - gamma * F(x))
        # Correction step
        x_next = project_C(x - gamma * F(y_hat))
        x = x_next
        history.append(x.copy())
        
    return np.array(history)

# Parameters
x0 = [1.0, 0.5]   # Initial point
gamma = 0.5        # Step size
max_iter = 30      # Number of iterations

# Run both methods
glp_path = glp_projection(x0, gamma, max_iter)
eg_path = extragradient(x0, gamma, max_iter)

# Calculate distance to solution (0,0)
glp_dist = np.linalg.norm(glp_path, axis=1)
eg_dist = np.linalg.norm(eg_path, axis=1)

# Print results
print("GLP Projection Method:")
print(f"{'Iter':<6}{'x':<10}{'y':<10}{'Distance to (0,0)':<15}")
for i, (point, dist) in enumerate(zip(glp_path, glp_dist)):
    print(f"{i:<6}{point[0]:<10.6f}{point[1]:<10.6f}{dist:<15.6f}")

print("\nExtragradient Method:")
print(f"{'Iter':<6}{'x':<10}{'y':<10}{'Distance to (0,0)':<15}")
for i, (point, dist) in enumerate(zip(eg_path, eg_dist)):
    print(f"{i:<6}{point[0]:<10.6f}{point[1]:<10.6f}{dist:<15.6f}")

GLP Projection Method:
Iter  x         y         Distance to (0,0)
0     1.000000  0.500000  1.118034       
1     1.000000  0.000000  1.000000       
2     1.000000  -0.500000 1.118034       
3     0.750000  -1.000000 1.250000       
4     0.250000  -1.000000 1.030776       
5     -0.250000 -1.000000 1.030776       
6     -0.750000 -0.875000 1.152443       
7     -1.000000 -0.500000 1.118034       
8     -1.000000 0.000000  1.000000       
9     -1.000000 0.500000  1.118034       
10    -0.750000 1.000000  1.250000       
11    -0.250000 1.000000  1.030776       
12    0.250000  1.000000  1.030776       
13    0.750000  0.875000  1.152443       
14    1.000000  0.500000  1.118034       
15    1.000000  0.000000  1.000000       
16    1.000000  -0.500000 1.118034       
17    0.750000  -1.000000 1.250000       
18    0.250000  -1.000000 1.030776       
19    -0.250000 -1.000000 1.030776       
20    -0.750000 -0.875000 1.152443       
21    -1.000000 -0.500000 1.118034       
22    -1.

## Key Insight

The extragradient method succeeds where GLP fails due to its prediction-correction mechanism:

Prediction step: Probes the operator at a future point

Correction step: Adjusts using updated information
This compensates for the lack of strong monotonicity and prevents oscillations.